**Báo cáo cuối kỳ môn học: PYTHON CHO KHOA HỌC DỮ LIỆU**

**Lớp 23TTH, Khoa Toán - Tin học, Trường Đại học Khoa học Tự nhiên, ĐHQG-HCM**

**Đề tài thực hiện:**
$$
\text{\textbf{USING DEEP LEARNING TO CLASSIFY ANIMAL AND HUMAN IMAGES}}
$$

**Giảng viên hướng dẫn: ThS. Hà Văn Thảo**

**Danh sách thành viên nhóm:**

1. 23110114 - Nguyễn Thị Hồng Thắm \
2. 23110123 - Lê Huỳnh Yến Vy \
3. 23110132 - Trần Nhật Anh

## GIỚI THIỆU

Object detection là một trong những chủ đề "nóng" trong deep learing bởi tính ứng dụng cao trong thực tiễn và nguồn dữ liệu dồi dào, dễ chuẩn bị. Một trong những thuật toán object detection nổi tiếng nhất là **YOLO**.

YOLO là mô hình mạng neuron tích chập (CNN) được sử dụng phổ biển để nhận dạng các đối tượng trong ảnh hoặc video. Điểm đặc biệt của mô hình này là có khả năng phát hiện tất cả các đối tượng trong một hình ảnh chỉ qua một lần lan truyền của CNN.

Các phương pháp truyền thống tách biệt bước đề xuất vùng và bước phân loại, YOLO xử lý đầu vào, vừa phân loại được các đối tượng, vừa dự đoán được vị trí của chúng trong một lần duy nhất.

YOLO có nghĩa là "You only look once", nghĩa là chỉ cần "nhìn" một lần là thuật toán đã có thể phát hiện được vật thể, cho thấy độ nhanh của thuật toán gần như là real-time.

Ứng dụng của YOLO cũng như nhiều thuật toán object detection khác, rất đa dạng: quản lý giao thông, đếm số sản phẩm trên băng chuyền nhà máy, đếm số vật nuôi trong chăn nuôi, phát hiện vật thể nguy hiểm (súng, dao,...), chấm công tự động,...

## TẠO MÔI TRƯỜNG ẢO VÀ KERNEL CHẠY NOTEBOOK (LINUX)

Dự án Python cần **môi trường ảo (virtual environment)** để tự cách ly, tránh xung đột phiên bản thư viện giữa các dự án. `venv` là môi trường ảo mà chúng ta sẽ sử dụng trong dự án này. Sau khi cài đặt `venv`, chúng ta di chuyển đường dẫn đến folder chứa dự án trong terminal và sử dụng lệnh sau để cài đặt môi trường ảo cho dự án:

`python -m venv .venv`

Trong đó, `.venv` là tên của folder chứa môi trường ảo của dự án, đồng thời nó cũng sẽ "đóng băng" phiên bản Python, pip và các thư viện sẽ được dùng trong dự án.

Kích hoạt môi trường ảo:

`source .venv/bin/activate`

Lúc này, phiên bản Python và `pip` được dùng là của môi trường ảo, các thư viện cài bằng `pip install` cũng chỉ ảnh hưởng trong `.venv`. Cách nhận biết đang ở môi trường ảo là promt terminal thường đổi thành `(.venv) user_name@machine:~` (nếu đang sử dụng Linux). Khi đã kích hoạt môi trường ảo, đảm bảo phiên bản Python và `pip` đã "đóng băng" trong đó, sử dụng lệnh:

`which python && which pip`

Nếu output có dạng `.../<project_name>/.venv/...` thì môi trường ảo đã được kích hoạt thành công.

Tiếp theo, tạo một kernel để chạy Jupyter Notebook. Cài đặt `ipykernel` để tạo kernel:

`python -m pip install ipykernel`

Sau khi cài đặt thành công, tiến hành tạo kernel để chạy file `.ipynb`:

`python -m ipykernel install --prefix .venv --name yolovenv --display-name "this_project"`

`--prefix .venv`: kernel mặc định không tự lưu vào `.venv`, thuộc tính này sẽ lưu kernel đã tạo vào `.venv`  
`--name yolovenv`: tên folder chứa kernel, ở đây tên folder là `yolovenv`. Kernel sẽ được lưu tại `.venv/share/jupyter/kernels/yolovenv/`  
`--display-name "this_project`: kernel sẽ hiển thị dưới tên `this_project` trong VS Code.

Khi đã tạo kernel, click vào biểu tượng kernel ở góc trên bên phải, chọn
$$
\text{Select Another Kernel} \rightarrow \text{Jupyter Kernel...} \rightarrow \text{this\_project}
$$

## KHAI BÁO THƯ VIỆN VÀ CHUẨN BỊ DỮ LIỆU

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import seaborn as sns
from ultralytics import YOLO

Data được tải về tại các nguồn sau: \
- https://www.kaggle.com/datasets/antoreepjana/animals-detection-images-dataset \
- https://www.kaggle.com/datasets/biancaferreira/african-wildlife \
- https://www.kaggle.com/datasets/wutheringwang/dog-face-detectionyolo-format \
- https://www.kaggle.com/datasets/samuelayman/cat-dataset
- https://universe.roboflow.com/labo-yolo/age-and-gender-xlnfj/dataset/3 \

Data sau khi được tải về sẽ được xử lý (gán lại class ID; phân loại thành các folder train, valid, test;...), sau đó được gộp thành một folder dataset duy nhất.

Sau khi hoàn tất xử lý dataset, chúng ta sẽ kiểm tra dataset có bị thiếu **nhãn dữ liệu** ứng với mỗi ảnh hay không.

In [2]:
BASE_DIR = Path("dataset/completed_dataset")
IMAGE_EXTS = {".jpg", ".jpeg", ".png"}
SPLITS = ["train", "valid", "test"]


def count_missing(split):
    images_dir = BASE_DIR / "images" / split
    labels_dir = BASE_DIR / "labels" / split

    total_images = 0
    missing = 0

    for img in images_dir.iterdir():
        if img.suffix.lower() not in IMAGE_EXTS:
            continue

        total_images += 1
        label_path = labels_dir / f"{img.stem}.txt"
        if not label_path.exists():
            missing += 1

    return total_images, missing


print(f"Checking dataset: {str(BASE_DIR)}\n")

grand_total = 0
grand_missing = 0
total_images_arr = np.array([])
missing_arr = np.array([])
percent_arr = np.array([])

for split in SPLITS:
    total_images, missing = count_missing(split)
    grand_total += total_images
    grand_missing += missing
    percent = (missing / total_images * 100) if total_images > 0 else 0

    total_images_arr = np.append(total_images_arr, total_images)
    missing_arr = np.append(missing_arr, missing)
    percent_arr = np.append(percent_arr, percent)

np_table = np.array([total_images_arr, missing_arr, percent_arr])
table = pd.DataFrame(np_table).transpose()
table.columns = ["total_images", "missing_labels", "missing_labels (%)"]
table.index = ["train", "valid", "test"]
print(table)

print("\n")
print(f"Total images: \t {grand_total}")
print(f"Missing: \t {grand_missing}")

Checking dataset: dataset/completed_dataset

       total_images  missing_labels  missing_labels (%)
train       20451.0             0.0                 0.0
valid        6078.0             0.0                 0.0
test         4637.0             0.0                 0.0


Total images: 	 31166
Missing: 	 0


Theo kết quả được in ra, dataset có tổng cộng 31166 hình ảnh và không có ảnh nào không có label. Như vậy, data đã đúng với format của YOLO.

## KHAI BÁO, HUẤN LUYỆN VÀ LƯU MÔ HÌNH

Hàm train model.

In [4]:
def train_model(model, dataset_path, project_path, project_name):
    model.train(
        data=dataset_path,
        epochs=50,  # Nếu ổn -> 50-80
        imgsz=416,  # Nếu ổn -> 640
        batch=8,  # Nếu ổn -> 8
        workers=4, # Nếu ổn -> 4
        device=0,  # Sử dụng GPU
        project=project_path,
        name=project_name,

        # Augmentation
        mosaic=0.25,
        mixup=0.0,
        copy_paste=0.0,

        # Optimization
        lr0=0.005,
        optimizer="SGD"
    )

Sử dụng model YOLO11s.

In [5]:
BASE_MODEL = "yolo11s.pt"

Huấn luyện.

In [8]:
DATASET_PATH = "dataset/completed_dataset/data.yaml"

# Lưu model tại "runs/yolo11s_custom/my_model" sau khi train
MODEL_PATH = Path("runs/yolo11s_custom/my_model3/weights/best.pt")
PROJECT_PATH = "runs/yolo11s_custom"
PROJECT_NAME = "my_model3"

# Huấn luyện mô hình với dataset
if MODEL_PATH.exists():
    print(f"Model has been trained already. It is being loaded again: {MODEL_PATH}")
    model = YOLO(str(MODEL_PATH))
else:
    print("Model hasn't been trained. Start training...")

    # Train model dựa trên model gốc là YOLO11s
    model = YOLO(BASE_MODEL)
    train_model(model, DATASET_PATH, PROJECT_PATH, PROJECT_NAME)

    # Load lại best.pt sau khi train, nếu không tìm thấy thì in ra lỗi
    assert MODEL_PATH.exists(), "Training finished but file best.pt not found"
    model = YOLO(str(MODEL_PATH))

    print("Training finished")

# Kiểm tra và khoá model với model gốc là YOLO11s
# assert model.model.yaml['name'] == 'yolo11s'
# model.info()

Model hasn't been trained. Start training...
New https://pypi.org/project/ultralytics/8.3.246 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.243 🚀 Python-3.14.2 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce GTX 1650, 3716MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset/completed_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=0.2

KeyboardInterrupt: 

## TEST